In [1]:
%reload_ext autoreload
%autoreload 2

"""
Align cells, pad to the same size, generate other three mirroring versions.
"""

import os
import sys
import time

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from data_manager import *
from metadata import *

import matplotlib.pyplot as plt
%matplotlib inline

from skimage.transform import rotate
from multiprocess import Pool

from cell_utilities import *

Setting environment for Gordon


No vtk


In [2]:
stack = 'MD594'

In [3]:
# all_shapes = []
# for sec in range(first_bs_section, last_bs_section+1):
#     masks = load_cell_data('mask', stack=stack, sec=sec)
#     if masks is not None:
#         shapes = [m.shape for m in masks.tolist()]
#         all_shapes += shapes

# common_shape = np.percentile(all_shapes, 99, axis=0).astype(np.int)
# print common_shape

common_shape = (57,62)

In [25]:
def align_pad_cells(stack, sec, common_shape):
    """
    Rotate cells so that its major axis is horizontal.
    Mirror in three ways.
    Output as hdf file.
    """

    sys.stderr.write('Processing section: %03d\n' % sec)

    try:
        blob_orientations = load_cell_data('orientation', stack, sec)
        blob_masks = load_cell_data('mask', stack, sec)
        blob_mask_centers = load_cell_data('mask_center', stack, sec)
    except:
        sys.stderr.write('ERROR: %d\n' % sec)
        return

    n_blobs = len(blob_masks)

    t = time.time()

    # Align major axis
    cells_aligned = [rotate(blob_masks[i], angle=-np.rad2deg(blob_orientations[i]),
                               center=blob_mask_centers[i],
                               cval=0.) > .5
                    for i in range(n_blobs)]
    cells_aligned_h = [m[:, ::-1] for m in cells_aligned]
    cells_aligned_v = [m[::-1, :] for m in cells_aligned]
    cells_aligned_d = [m[::-1, ::-1] for m in cells_aligned]

    sys.stderr.write('Rotate cell masks: %.2f\n' % (time.time() - t)) # 60s

#     save_hdf_v2(cells_aligned, get_cell_data_filepath('cells_aligned', stack, sec))
#     save_hdf_v2(cells_aligned_h, get_cell_data_filepath('cells_aligned_h', stack, sec))
#     save_hdf_v2(cells_aligned_v, get_cell_data_filepath('cells_aligned_v', stack, sec))
#     save_hdf_v2(cells_aligned_d, get_cell_data_filepath('cells_aligned_d', stack, sec))
    
    
    cells_padded = pad_patches_to_same_size(cells_aligned, pad_value=0, keep_center=True, common_shape=common_shape)
    bp.pack_ndarray_file(np.asarray(cells_padded), get_cell_data_filepath('cells_aligned_padded', stack, sec))
    
    cells_padded_h = pad_patches_to_same_size(cells_aligned_h, pad_value=0, keep_center=True, common_shape=common_shape)
    bp.pack_ndarray_file(np.asarray(cells_padded_h), get_cell_data_filepath('cells_aligned_padded_h', stack, sec))
    
    cells_padded_v = pad_patches_to_same_size(cells_aligned_v, pad_value=0, keep_center=True, common_shape=common_shape)
    bp.pack_ndarray_file(np.asarray(cells_padded_v), get_cell_data_filepath('cells_aligned_padded_v', stack, sec))
    
    cells_padded_d = pad_patches_to_same_size(cells_aligned_d, pad_value=0, keep_center=True, common_shape=common_shape)
    bp.pack_ndarray_file(np.asarray(cells_padded_d), get_cell_data_filepath('cells_aligned_padded_d', stack, sec))

In [31]:
first_bs_section, last_bs_section = metadata_cache['section_limits'][stack]

In [ ]:
t = time.time()

pool = Pool(15)
pool.map(lambda sec: align_pad_cells(stack, sec, common_shape=common_shape), 
         range(first_bs_section, last_bs_section+1))
pool.close()
pool.join()

sys.stderr.write('Align cells overall: %.2f\n' % (time.time() - t)) # 330 seconds

Processing section: 097
Processing section: 092
Processing section: 102
Processing section: 107
Processing section: 112
Processing section: 117
Processing section: 122
Processing section: 127
Processing section: 132
Processing section: 137
Processing section: 142
Processing section: 147
Processing section: 152
Processing section: 157
Processing section: 162
